## Databricks - Genomics

<br>

This may help to see the big picture:
*  https://en.wikipedia.org/wiki/1000_Genomes_Project

<div class="alert alert-block alert-warning">
<b>Note:</b> There are a **lot** of dependencies to make this notebook work.  In the Appendix I list all of them...
</div>

This notebook is designed to provide a broad overview of Hail's functionality, with emphasis on the functionality to manipulate and query a genetic dataset. 

We walk through a genome-wide SNP association test, and demonstrate the need to control for confounding caused by population stratification.  Parts of this are taken from Hail's online tutorial series: 


*  https://hail.is/docs/0.2/tutorials/07-matrixtable.html

READ:  Remember, you do NOT need to initiate a SparkSession (context) like you do in normal vanilla Apache Spark.  It is pre-built for you when you open up a Spark notebook in Databricks.  So don't re-create or double create spark sessions, it will fail...

In [7]:
import hail as hl
import hail.expr.aggregators as agg
hl.init(sc, idempotent=True)

Running on Apache Spark version 2.4.3
SparkUI available at http://10.22.233.230:43530
Welcome to
 __ __ <>__
 / /_/ /__ __/ /
 / __ / _ `/ / /
 /_/ /_/\_,_/_/_/ version 0.2.12-9409c0635781
LOGGING: writing to /databricks/driver/hail-20190816-2133-0.2.12-9409c0635781.log

In [8]:
# Looks very similar to Apache Spark when it initiates...

In [9]:
# side note:   it took me 42 days to make this work right.
# my advice:   sometimes its ok to ask for help.  
# advice2:     if your not first, you're last. 
# advice3:     if you are new to Apache Spark, don't start with a crazy complex genomics sequencing project as your first dive into it

In [10]:
from pprint import pprint
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import Span
output_notebook()

In [11]:
from bokeh.embed import components, file_html
from bokeh.resources import CDN

def displayBokeh(p):
  html = file_html(p, CDN, "Chart")
  # display this html
  displayHTML(html)

In [12]:
# --- core files we will use for this --- 
vcf_path = '/databricks-datasets/hail/data-001/1kg_sample.vcf.bgz'
annotation_path = '/databricks-datasets/hail/data-001/1kg_annotations.txt'

In [13]:
# --- databricks actually comes with some genomics raw data for you to mess with ---
display(dbutils.fs.ls("/databricks-datasets/hail/data-001"))

path,name,size
dbfs:/databricks-datasets/hail/data-001/1kg_annotations.txt,1kg_annotations.txt,22784
dbfs:/databricks-datasets/hail/data-001/1kg_sample.vcf.bgz,1kg_sample.vcf.bgz,39767725
dbfs:/databricks-datasets/hail/data-001/purcell5k.interval_list,purcell5k.interval_list,192078


### Loading data from disk

Hail has its own internal data representation, called a MatrixTable. This is both an on-disk file format and a [Python object](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable). Here, we read a MatrixTable from DBFS.

In [15]:
mt = hl.import_vcf(vcf_path)

### Data 

It's important to have easy ways to slice, dice, query, and summarize a dataset. Some of these methods are demonstrated below.

The [rows](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.rows) method can be used to get a table with all the row fields in our MatrixTable. 

We can use `rows` along with [select](https://hail.is/docs/devel/hail.Table.html#hail.Table.select) to pull out 5 variants. The `select` method takes either a string refering to a field name in the table, or a Hail [Expression](https://hail.is/docs/devel/expr/expression.html#hail.expr.expression.Expression). Here, we leave the arguments blank to keep only the row key fields, `locus` and `alleles`.

Use the `show` method to display the variants.

In [18]:
mt.rows().select().show(5)

¬í w72019-08-16 21:33:50 Hail: INFO: Coerced sorted dataset
2019-08-16 21:33:50 Hail: INFO: Coerced sorted dataset
+---------------+------------+
 locus | alleles |
+---------------+------------+
 locus<GRCh37> | array<str> |
+---------------+------------+
 1:904165 | ["G","A"] |
 1:909917 | ["G","A"] |
 1:986963 | ["C","T"] |
 1:1563691 | ["T","G"] |
 1:1707740 | ["T","G"] |
+---------------+------------+
showing top 5 rows

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-856786140303328> in <module> () 
 ----> 1 mt . row_value ( ) 

 TypeError : 'StructExpression' object is not callable

Here is how to peek at the first few sample IDs:

In [21]:
mt.s.show(5)

+-----------+
 s |
+-----------+
 str |
+-----------+
 "HG00096" |
 "HG00100" |
 "HG00105" |
 "HG00114" |
 "HG00115" |
+-----------+
showing top 5 rows

To look at the first few genotype calls, we can use [entries](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.entries) along with `select` and `take`. The `take` method collects the first n rows into a list. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format. 

Try changing `take` to `show` in the cell below.

In [23]:
mt.entry.take(5)

Out[9]: [Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=12, PL=[0, 12, 147]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=12, PL=[0, 12, 155]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=23, PL=[0, 23, 230]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[3, 0], DP=3, GQ=9, PL=[0, 9, 108]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=14, PL=[0, 14, 136])]

## Adding column fields

A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. Column fields are where you'll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis. 

In this tutorial, we demonstrate how to take a text file and use it to annotate the columns in a MatrixTable.

The file provided contains the sample ID, the population and "super-population" designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

This file can be imported into Hail with [import_table](https://hail.is/docs/devel/methods/impex.html#hail.methods.import_table). This method produces a [Table](https://hail.is/docs/devel/hail.Table.html#hail.Table) object. Think of this as a Pandas or R dataframe that isn't limited by the memory on your machine -- behind the scenes, it's distributed with Spark.

In [27]:
table = (hl.import_table(annotation_path, impute=True)
         .key_by('Sample'))

wE2019-08-16 21:33:51 Hail: INFO: Reading table to impute column types
2019-08-16 21:33:51 Hail: INFO: Reading table to impute column types
¬í wE2019-08-16 21:33:51 Hail: INFO: Reading table to impute column types
z  2019-08-16 21:33:52 Hail: INFO: Finished type imputation
z  2019-08-16 21:33:52 Hail: INFO: Finished type imputation
2019-08-16 21:33:52 Hail: INFO: Finished type imputation
 Loading column 'Sample' as type 'str' (imputed)
 Loading column 'Population' as type 'str' (imputed)
 Loading column 'SuperPopulation' as type 'str' (imputed)
 Loading column 'Sample' as type 'str' (imputed)
 Loading column 'Population' as type 'str' (imputed)
 Loading column 'SuperPopulation' as type 'str' (imputed)
 Loading column 'Sample' as type 'str' (imputed)
 Loading column 'Population' as type 'str' (imputed)
 Loading column 'SuperPopulation' as type 'str' (imputed)
 Loading column 'isFemale' as type 'bool' (imputed)
 Loading column 'PurpleHair' as type 'bool' (imputed)
 Loading column 'CaffeineConsumption' as type 'float64' (imputed)
 Loading column 'isFemale' as type 'bool' (imputed)
 Loading column 'PurpleHair' as type 'bool' (imputed)
 Loading column 'CaffeineConsumption' as type 'float64' (imputed)
 Loading column 'isFemale' as type 'bool' (imputed)
 Loading column 'PurpleHair' as type 'bool' (imputed)
 Loading column 'CaffeineConsumption' as type 'float64' (imputed)

A good way to peek at the structure of a `Table` is to look at its `schema`.

In [29]:
table.describe()

----------------------------------------
Global fields:
 None
----------------------------------------
Row fields:
 'Sample': str 
 'Population': str 
 'SuperPopulation': str 
 'isFemale': bool 
 'PurpleHair': bool 
 'CaffeineConsumption': float64 
----------------------------------------
Key: ['Sample']
----------------------------------------

To peek at the first few values, use the `show` method:

In [31]:
table.show(width=100)

+-----------+------------+-----------------+----------+------------+---------------------+
 Sample | Population | SuperPopulation | isFemale | PurpleHair | CaffeineConsumption |
+-----------+------------+-----------------+----------+------------+---------------------+
 str | str | str | bool | bool | float64 |
+-----------+------------+-----------------+----------+------------+---------------------+
 "HG00096" | "GBR" | "EUR" | false | false | 7.70e+01 |
 "HG00100" | "GBR" | "EUR" | true | false | 5.90e+01 |
 "HG00105" | "GBR" | "EUR" | false | false | 7.70e+01 |
 "HG00114" | "GBR" | "EUR" | false | true | 7.20e+01 |
 "HG00115" | "GBR" | "EUR" | false | true | 7.10e+01 |
 "HG00116" | "GBR" | "EUR" | false | false | 7.10e+01 |
 "HG00119" | "GBR" | "EUR" | false | true | 7.00e+01 |
 "HG00120" | "GBR" | "EUR" | true | true | 6.80e+01 |
 "HG00128" | "GBR" | "EUR" | true | true | 6.70e+01 |
 "HG00131" | "GBR" | "EUR" | false | false | 7.20e+01 |
+-----------+------------+-----------------+----------+------------+---------------------+
showing top 10 rows

Now we'll use this table to add sample annotations to our dataset, storing the annotations in column fields in our MatrixTable. First, we'll print the existing column schema:

In [33]:
print(mt.col.dtype)

struct{s: str}

We use the [annotate_cols](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) method to join the table with the MatrixTable containing our dataset.

In [35]:
mt = mt.annotate_cols(**table[mt.s])

In [36]:
print(mt.col.dtype)

struct{s: str, Population: str, SuperPopulation: str, isFemale: bool, PurpleHair: bool, CaffeineConsumption: float64}

In [37]:
print(mt.col.dtype.pretty())

struct {
 s: str, 
 Population: str, 
 SuperPopulation: str, 
 isFemale: bool, 
 PurpleHair: bool, 
 CaffeineConsumption: float64
}

## Query functions and the Hail Expression Language

Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments.

We will start by looking at some statistics of the information in our table. The [aggregate](https://hail.is/docs/devel/hail.Table.html#hail.Table.aggregate) method can be used to aggregate over rows of the table.

`counter` is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.

In [40]:
pprint(table.aggregate(agg.counter(table.SuperPopulation)))

w72019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
2019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
w72019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
{'AFR': 193, 'AMR': 96, 'EAS': 149, 'EUR': 140, 'SAS': 132}

`stats` is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [42]:
pprint(table.aggregate(agg.stats(table.CaffeineConsumption)))

w72019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
w72019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
2019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
{'max': 80.0,
 'mean': 67.1605633802817,
 'min': 55.0,
 'n': 710,
 'stdev': 4.5405366337074415,
 'sum': 47684.00000000001}

However, these metrics aren't perfectly representative of the samples in our dataset. Here's why:

In [44]:
table.count()

Out[19]: 710

In [45]:
mt.count_cols()

Out[20]: 710

Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use [aggregate_cols](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) to get the metrics for only the samples in our dataset.

In [47]:
mt.aggregate_cols(agg.counter(mt.SuperPopulation))

w72019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
w72019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
2019-08-16 21:33:53 Hail: INFO: Coerced sorted dataset
Out[21]: {'AFR': 193, 'EAS': 149, 'AMR': 96, 'SAS': 132, 'EUR': 140}

In [48]:
pprint(mt.aggregate_cols(agg.stats(mt.CaffeineConsumption)))

w72019-08-16 21:33:54 Hail: INFO: Coerced sorted dataset
w72019-08-16 21:33:54 Hail: INFO: Coerced sorted dataset
2019-08-16 21:33:54 Hail: INFO: Coerced sorted dataset
{'max': 80.0,
 'mean': 67.16056338028172,
 'min': 55.0,
 'n': 710,
 'stdev': 4.54053663370744,
 'sum': 47684.000000000015}

The functionality demonstrated in the last few cells isn't anything especially new: it's certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like `awk`. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants. 

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base). 

To do this, we need to get the alternate allele of each variant and then count the occurences of each unique ref/alt pair. This can be done with Hail's `counter` method.

In [50]:
snp_counts = mt.aggregate_rows(agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
pprint(snp_counts)

{Struct(ref='T', alt='G'): 468,
 Struct(ref='T', alt='A'): 79,
 Struct(ref='T', alt='C'): 1879,
 Struct(ref='G', alt='T'): 480,
 Struct(ref='A', alt='G'): 1944,
 Struct(ref='C', alt='A'): 496,
 Struct(ref='C', alt='T'): 2436,
 Struct(ref='A', alt='C'): 454,
 Struct(ref='G', alt='A'): 2387,
 Struct(ref='C', alt='G'): 150,
 Struct(ref='G', alt='C'): 112,
 Struct(ref='A', alt='T'): 76}

We can list the counts in descending order using Python's Counter class.

In [52]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

Out[24]: [(Struct(ref='C', alt='T'), 2436),
 (Struct(ref='G', alt='A'), 2387),
 (Struct(ref='A', alt='G'), 1944),
 (Struct(ref='T', alt='C'), 1879),
 (Struct(ref='C', alt='A'), 496),
 (Struct(ref='G', alt='T'), 480),
 (Struct(ref='T', alt='G'), 468),
 (Struct(ref='A', alt='C'), 454),
 (Struct(ref='C', alt='G'), 150),
 (Struct(ref='G', alt='C'), 112),
 (Struct(ref='T', alt='A'), 79),
 (Struct(ref='A', alt='T'), 76)]

It's nice to see that we can actually uncover something biological from this small dataset: we see that these frequencies come in pairs. C/T and G/A are actually the same mutation, just viewed from from opposite strands. Likewise, T/A and A/T are the same mutation on opposite strands. There's a 30x difference between the frequency of C/T and A/T SNPs. Why?

The same Python, R, and Unix tools could do this work as well, but we're starting to hit a wall - the latest [gnomAD release](http://gnomad.broadinstitute.org/) publishes about 250 million variants, and that won't fit in memory on a single computer.

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 284 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about **5 trillion** unique genotypes.

Hail plotting functions allow Hail fields as arguments, so we can pass in the DP field directly here. If the range and bins arguments are not set, this function will compute the range based on minimum and maximum values of the field and use the default 50 bins.

In [56]:
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

## Quality Control

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no "push-button" solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [sample_qc](https://hail.is/docs/devel/methods/genetics.html#hail.methods.sample_qc) method, which produces a set of useful metrics and stores them in a column field.

In [59]:
mt.col.describe()

--------------------------------------------------------
Type:
 struct {
 s: str, 
 Population: str, 
 SuperPopulation: str, 
 isFemale: bool, 
 PurpleHair: bool, 
 CaffeineConsumption: float64
 }
--------------------------------------------------------
Source:
 <hail.matrixtable.MatrixTable object at 0x7f502c1382b0>
Index:
 ['column']
--------------------------------------------------------

In [60]:
mt = hl.sample_qc(mt)

In [61]:
mt.col.describe()

--------------------------------------------------------
Type:
 struct {
 s: str, 
 Population: str, 
 SuperPopulation: str, 
 isFemale: bool, 
 PurpleHair: bool, 
 CaffeineConsumption: float64, 
 sample_qc: struct {
 dp_stats: struct {
 mean: float64, 
 stdev: float64, 
 min: float64, 
 max: float64
 }, 
 gq_stats: struct {
 mean: float64, 
 stdev: float64, 
 min: float64, 
 max: float64
 }, 
 call_rate: float64, 
 n_called: int64, 
 n_not_called: int64, 
 n_hom_ref: int64, 
 n_het: int64, 
 n_hom_var: int64, 
 n_non_ref: int64, 
 n_singleton: int64, 
 n_snp: int64, 
 n_insertion: int64, 
 n_deletion: int64, 
 n_transition: int64, 
 n_transversion: int64, 
 n_star: int64, 
 r_ti_tv: float64, 
 r_het_hom_var: float64, 
 r_insertion_deletion: float64
 }
 }
--------------------------------------------------------
Source:
 <hail.matrixtable.MatrixTable object at 0x7f502b35d5c0>
Index:
 ['column']
--------------------------------------------------------

Plotting the QC metrics is a good place to start.

In [63]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

In [64]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

Often, these metrics are correlated.

In [66]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

Removing outliers from the dataset will generally improve association results. We can draw lines on the above plot to indicate outlier cuts.

In [68]:
p.renderers.extend(
    [Span(location=0.97, dimension='width', line_color='black', line_width=1),
     Span(location=4, dimension='height', line_color='black', line_width=1)])
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

We'll want to remove all samples that fall in the bottom left quadrant. 

It's easy to filter when we've got the cutoff values decided:

In [70]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

w72019-08-16 21:34:07 Hail: INFO: Coerced sorted dataset
w72019-08-16 21:34:07 Hail: INFO: Coerced sorted dataset
2019-08-16 21:34:07 Hail: INFO: Coerced sorted dataset
After filter, 645/284 samples remain.

Next is genotype QC. To start, we'll print the post-sample-QC call rate. It's actually gone _up_ since the initial summary - dropping low-quality samples disproportionally removed missing genotypes.

Import the `hail.expr.functions` module.

In [72]:
call_rate = mt.aggregate_entries(agg.fraction(hl.is_defined(mt.GT)))
print('pre QC call rate is %.3f' % call_rate)

pre QC call rate is 0.992

It's a good idea to filter out genotypes where the reads aren't where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In [74]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

mt = mt.filter_entries(filter_condition_ab)

In [75]:
post_qc_call_rate = mt.aggregate_entries(agg.fraction(hl.is_defined(mt.GT)))
print('post QC call rate is %.3f' % post_qc_call_rate)

post QC call rate is 1.000

Variant QC is a bit more of the same: we can use the [variant_qc](https://hail.is/docs/devel/methods/genetics.html?highlight=variant%20qc#hail.methods.variant_qc) method to produce a variety of useful statistics, plot them, and filter.

In [77]:
mt.row.describe()

--------------------------------------------------------
Type:
 struct {
 locus: locus<GRCh37>, 
 alleles: array<str>, 
 rsid: str, 
 qual: float64, 
 filters: set<str>, 
 info: struct {
 AC: array<int32>, 
 AF: array<float64>, 
 AN: int32, 
 BaseQRankSum: float64, 
 ClippingRankSum: float64, 
 DP: int32, 
 DS: bool, 
 FS: float64, 
 HaplotypeScore: float64, 
 InbreedingCoeff: float64, 
 MLEAC: array<int32>, 
 MLEAF: array<float64>, 
 MQ: float64, 
 MQ0: int32, 
 MQRankSum: float64, 
 QD: float64, 
 ReadPosRankSum: float64, 
 set: str
 }
 }
--------------------------------------------------------
Source:
 <hail.matrixtable.MatrixTable object at 0x7f502b346898>
Index:
 ['row']
--------------------------------------------------------

The [cache](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.cache) is used to optimize some of the downstream operations.

In [79]:
mt = hl.variant_qc(mt).cache()

w72019-08-16 21:34:21 Hail: INFO: Coerced sorted dataset
w72019-08-16 21:34:21 Hail: INFO: Coerced sorted dataset
2019-08-16 21:34:21 Hail: INFO: Coerced sorted dataset

In [80]:
mt.row.describe()

--------------------------------------------------------
Type:
 struct {
 locus: locus<GRCh37>, 
 alleles: array<str>, 
 rsid: str, 
 qual: float64, 
 filters: set<str>, 
 info: struct {
 AC: array<int32>, 
 AF: array<float64>, 
 AN: int32, 
 BaseQRankSum: float64, 
 ClippingRankSum: float64, 
 DP: int32, 
 DS: bool, 
 FS: float64, 
 HaplotypeScore: float64, 
 InbreedingCoeff: float64, 
 MLEAC: array<int32>, 
 MLEAF: array<float64>, 
 MQ: float64, 
 MQ0: int32, 
 MQRankSum: float64, 
 QD: float64, 
 ReadPosRankSum: float64, 
 set: str
 }, 
 variant_qc: struct {
 AC: array<int32>, 
 AF: array<float64>, 
 AN: int32, 
 homozygote_count: array<int32>, 
 dp_stats: struct {
 mean: float64, 
 stdev: float64, 
 min: float64, 
 max: float64
 }, 
 gq_stats: struct {
 mean: float64, 
 stdev: float64, 
 min: float64, 
 max: float64
 }, 
 n_called: int64, 
 n_not_called: int64, 
 call_rate: float32, 
 n_het: int64, 
 n_non_ref: int64, 
 het_freq_hwe: float64, 
 p_value_hwe: float64
 }
 }
--------------------------------------------------------
Source:
 <hail.matrixtable.MatrixTable object at 0x7f502b36c9e8>
Index:
 ['row']
--------------------------------------------------------

In [81]:
qc_table = mt.rows()
qc_table = qc_table.select(gq_mean = qc_table.variant_qc.gq_stats.mean,
                           call_rate = qc_table.variant_qc.call_rate,
                           p_value_hwe = qc_table.variant_qc.p_value_hwe,
                           AAF = qc_table.variant_qc.AF[1],)

p1 = hl.plot.histogram(qc_table.gq_mean, range=(10,80), legend='Variant Mean GQ')

p2 = hl.plot.histogram(qc_table.AAF, range=(0,1), legend='Alternate Allele Frequency')

p3 = hl.plot.histogram(qc_table.call_rate, range=(.5,1), legend='Variant Call Rate')

p4 = hl.plot.histogram(qc_table.p_value_hwe, range=(0,1), legend='Hardy-Weinberg Equilibrium p-value')

g = gridplot([[p1, p2], [p3, p4]])
displayBokeh(g)

<!DOCTYPE html>
 
 
 
 Chart

These statistics actually look pretty good: we don't need to filter this dataset. Most datasets require thoughtful quality control, though. The [filter_rows](https://hail.is/docs/devel/hail.MatrixTable.html#hail.MatrixTable.filter_rows) method can help!

## Let's do a GWAS!

First, we need to restrict to variants that are : 

 - common (we'll use a cutoff of 1%)
 - uncorrelated (not in linkage disequilibrium)

In [84]:
common_mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

In [85]:
print('Samples: %d  Variants: %d' % (common_mt.count_cols(), common_mt.count_rows()))

Samples: 645 Variants: 9181

These filters removed about 15% of sites (we started with a bit over 10,000). This is _NOT_ representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we'd expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we've already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [87]:
gwas = hl.linear_regression_rows(y=common_mt.CaffeineConsumption, x=common_mt.GT.n_alt_alleles(), covariates=[1.0])
gwas.row.describe()

w£2019-08-16 21:34:40 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
2019-08-16 21:34:40 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 1 additional covariate...
w£2019-08-16 21:34:40 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 1 additional covariate...
 with input variable x, and 1 additional covariate...
--------------------------------------------------------
Type:
 struct {
 locus: locus<GRCh37>, 
 alleles: array<str>, 
 n: int32, 
 sum_x: float64, 
 y_transpose_x: float64, 
 beta: float64, 
 standard_error: float64, 
 t_stat: float64, 
 p_value: float64
 }
--------------------------------------------------------
Source:
 <hail.table.Table object at 0x7f502b22bc18>
Index:
 ['row']
--------------------------------------------------------

Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

Hail makes it easy to make a [Q-Q (quantile-quantile) plot](https://en.wikipedia.org/wiki/Q–Q_plot).

In [89]:
p = hl.plot.qq(gwas.p_value)
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there's a confounder.

We didn't tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a [stratified](https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression. 

The [linear_regression](https://hail.is/docs/devel/methods/stats.html#hail.methods.linear_regression) method can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don't have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The [pca](https://hail.is/docs/devel/methods/stats.html#hail.methods.pca) method produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The [hwe_normalized_pca](https://hail.is/docs/devel/methods/genetics.html#hail.methods.hwe_normalized_pca) method does the same, using HWE-normalized genotypes for the PCA.

In [91]:
pca_eigenvalues, pca_scores, _ = hl.hwe_normalized_pca(common_mt.GT)

wU2019-08-16 21:34:44 Hail: INFO: hwe_normalized_pca: running PCA using 9175 variants.
wU2019-08-16 21:34:44 Hail: INFO: hwe_normalized_pca: running PCA using 9175 variants.
2019-08-16 21:34:44 Hail: INFO: hwe_normalized_pca: running PCA using 9175 variants.
wG2019-08-16 21:34:46 Hail: INFO: pca: running PCA with 10 components...
2019-08-16 21:34:46 Hail: INFO: pca: running PCA with 10 components...
wG2019-08-16 21:34:46 Hail: INFO: pca: running PCA with 10 components...

In [92]:
pprint(pca_eigenvalues)

[57.996643048591324,
 26.689264215387045,
 8.616338891239343,
 6.641941578683806,
 2.1549928666006473,
 2.0028643498637253,
 1.7949104672357343,
 1.7618763024658695,
 1.717872646606727,
 1.710513112125637]

In [93]:
pca_scores.show(5, width=100)

+-----------+
 s |
+-----------+
 str |
+-----------+
 "HG00096" |
 "HG00100" |
 "HG00105" |
 "HG00115" |
 "HG00116" |
+-----------+

+-------------------------------------------------------------------------------------------------+
 scores |
+-------------------------------------------------------------------------------------------------+
 array<float64> |
+-------------------------------------------------------------------------------------------------+
 [1.51e-01,2.72e-01,2.14e-02,-1.39e-01,3.27e-02,-2.50e-02,-4.60e-02,1.02e-01,6.19e-02,1.09e-02] |
 [1.61e-01,2.88e-01,1.85e-02,-1.08e-01,-1.16e-02,-2.14e-03,2.71e-02,4.34e-02,1.51e-02,4.37e-02] |
 [1.51e-01,2.77e-01,3.17e-02,-1.15e-01,4.53e-03,-9.80e-03,4.24e-03,1.80e-02,-5.03e-02,-3.40e-02] |
 [1.51e-01,2.98e-01,8.82e-03,-7.76e-02,8.07e-03,1.82e-02,-2.96e-02,-1.16e-02,-4.57e-02,4.04e-02] |
 [1.43e-01,2.88e-01,4.43e-02,-1.24e-01,8.55e-03,-8.00e-03,-5.02e-02,1.25e-02,4.22e-02,-1.13e-01] |
+-------------------------------------------------------------------------------------------------+
showing top 5 rows

Now that we've got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [95]:
p = hl.plot.scatter(pca_scores.scores[0], pca_scores.scores[1],
                    label=common_mt.cols()[pca_scores.s].SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

Now we can rerun our linear regression, controlling for sample sex and the first few principal components. We'll do this with input variable the number of alternate alleles as before, and again with input variable the genotype dosage derived from the PL field.

In [97]:
cmt = common_mt.annotate_cols(pca = pca_scores[common_mt.s])

linear_regression_results = hl.linear_regression_rows(
    y=cmt.CaffeineConsumption, x=cmt.GT.n_alt_alleles(),
    covariates=[1.0, cmt.isFemale, cmt.pca.scores[0], cmt.pca.scores[1], cmt.pca.scores[2]])

w¤2019-08-16 21:34:58 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
w¤2019-08-16 21:34:58 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 5 additional covariates...
2019-08-16 21:34:58 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 5 additional covariates...
 with input variable x, and 5 additional covariates...

In [98]:
p = hl.plot.qq(linear_regression_results.p_value)
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

We can also use the genotype dosage (the probability-weighted expected number of alternate alleles) instead of the hard call in our regression

In [100]:
def plDosage(pl):
    linear_scaled = pl.map(lambda x: 10 ** (-x / 10))
    pl_sum = hl.sum(linear_scaled)
    normed = linear_scaled / pl_sum
    return 1 * normed[1] + 2 * normed[2]

In [101]:
linear_regression_results = hl.linear_regression_rows(
    y=cmt.CaffeineConsumption, x=plDosage(cmt.PL),
    covariates=[1.0, cmt.isFemale, cmt.pca.scores[0], cmt.pca.scores[1], cmt.pca.scores[2]])

w¤2019-08-16 21:35:01 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 5 additional covariates...
w¤2019-08-16 21:35:01 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 5 additional covariates...
2019-08-16 21:35:01 Hail: INFO: linear_regression_rows: running on 645 samples for 1 response variable y,
 with input variable x, and 5 additional covariates...

In [102]:
p = hl.plot.qq(linear_regression_results.p_value)
displayBokeh(p)

<!DOCTYPE html>
 
 
 
 Chart

That's more like it! We may not be publishing ten new coffee-drinking loci in _Nature_, but we shouldn't expect to find anything but the strongest signals from a dataset of 284 individuals anyway.

## Rare variant analysis

Here we'll demonstrate how one can use the expression language to group and count by any arbitrary properties in row and column fields. Hail also implements the sequence kernel association test (SKAT).

In [105]:
entries = mt.entries()
results = (entries.group_by(pop = entries.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = agg.count_where(entries.GT.is_het())))

w½2019-08-16 21:35:07 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
w½2019-08-16 21:35:07 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
 To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'
 To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'
2019-08-16 21:35:07 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
 To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'

In [106]:
results.show()

wO2019-08-16 21:35:21 Hail: INFO: Ordering unsorted dataset with network shuffle
wO2019-08-16 21:35:21 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-16 21:35:21 Hail: INFO: Ordering unsorted dataset with network shuffle
wO2019-08-16 21:35:35 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-16 21:35:35 Hail: INFO: Ordering unsorted dataset with network shuffle
wO2019-08-16 21:35:35 Hail: INFO: Ordering unsorted dataset with network shuffle
+-------+------------+-------+
 pop | chromosome | n_het |
+-------+------------+-------+
 str | str | int64 |
+-------+------------+-------+
 "AFR" | "1" | 35272 |
 "AFR" | "10" | 22613 |
 "AFR" | "11" | 21919 |
 "AFR" | "12" | 21865 |
 "AFR" | "13" | 14398 |
 "AFR" | "14" | 13803 |
 "AFR" | "15" | 12822 |
 "AFR" | "16" | 15108 |
 "AFR" | "17" | 12606 |
 "AFR" | "18" | 12928 |
+-------+------------+-------+
showing top 10 rows

What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?

In [108]:
entries = entries.annotate(maf_bin = hl.cond(entries.info.AF[0]<0.01, "< 1%", 
                             hl.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.PurpleHair)
      .aggregate(mean_gq = agg.stats(entries.GQ).mean, 
                 mean_dp = agg.stats(entries.DP).mean))

In [109]:
results2.show()

wO2019-08-16 21:35:50 Hail: INFO: Ordering unsorted dataset with network shuffle
wO2019-08-16 21:35:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-16 21:35:50 Hail: INFO: Ordering unsorted dataset with network shuffle
wO2019-08-16 21:36:05 Hail: INFO: Ordering unsorted dataset with network shuffle
wO2019-08-16 21:36:05 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-08-16 21:36:05 Hail: INFO: Ordering unsorted dataset with network shuffle
+---------+-------------+----------+----------+
 af_bin | purple_hair | mean_gq | mean_dp |
+---------+-------------+----------+----------+
 str | bool | float64 | float64 |
+---------+-------------+----------+----------+
 "1%-5%" | false | 2.38e+01 | 7.27e+00 |
 "1%-5%" | true | 2.38e+01 | 7.27e+00 |
 "< 1%" | false | 2.27e+01 | 7.36e+00 |
 "< 1%" | true | 2.27e+01 | 7.38e+00 |
 ">5%" | false | 3.57e+01 | 7.40e+00 |
 ">5%" | true | 3.57e+01 | 7.40e+00 |
+---------+-------------+----------+----------+

We've shown that it's easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

 - Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint
 - Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease

<br>
<br>

# Tutorial on MatrixTable

A `MatrixTable` has four kinds of fields:
*  global fields
*  row fields
*  column fields
*  entry fields

In [114]:
from bokeh.io import output_notebook, show
output_notebook()

In [115]:
%fs
ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks/,databricks/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/ml/,ml/,0
dbfs:/tmp/,tmp/,0
dbfs:/user/,user/,0


Ok, if you want to get really really tricky here, you could go to the Hail Github location, where they actually have independent files that are scripts in python to pull down enormous amounts of genomics data.  Think a million wgets pulling down data, if you do it right...

* https://github.com/hail-is/hail/tree/master/datasets/load

In [118]:
print("""
Example:  We could infiltrate this...

ht_samples = hl.read_table('gs://hail-datasets-hail-data/1000_Genomes_phase3_samples.ht')
ht_relationships = hl.read_table('gs://hail-datasets-hail-data/1000_Genomes_phase3_sample_relationships.ht')

mt = hl.import_vcf(
    'gs://hail-datasets-raw-data/1000_Genomes/1000_Genomes_phase3_chr{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22}_GRCh37.vcf.bgz',
    reference_genome='GRCh37')

mt = mt.annotate_cols(**ht_samples[mt.s])
mt = mt.annotate_cols(**ht_relationships[mt.s])

mt_split = hl.split_multi(mt)
mt_split = mt_split.select_entries(GT=hl.downcode(mt_split.GT, mt_split.a_index))
mt_split = mt_split.annotate_rows(info=hl.struct(
    CIEND=mt_split.info.CIEND[mt_split.a_index - 1],
    CIPOS=mt_split.info.CIPOS[mt_split.a_index - 1],
    CS=mt_split.info.CS,
    END=mt_split.info.END,
    IMPRECISE=mt_split.info.IMPRECISE,
    MC=mt_split.info.MC,
    MEINFO=mt_split.info.MEINFO,
    MEND=mt_split.info.MEND,
    MLEN=mt_split.info.MLEN,
    MSTART=mt_split.info.MSTART,
    SVLEN=mt_split.info.SVLEN,
    SVTYPE=mt_split.info.SVTYPE,
    TSD=mt_split.info.TSD,
    AC=mt_split.info.AC[mt_split.a_index - 1],
    AF=mt_split.info.AF[mt_split.a_index - 1],
    NS=mt_split.info.NS,
    AN=mt_split.info.AN,
    EAS_AF=mt_split.info.EAS_AF[mt_split.a_index - 1],
    EUR_AF=mt_split.info.EUR_AF[mt_split.a_index - 1],
    AFR_AF=mt_split.info.AFR_AF[mt_split.a_index - 1],
    AMR_AF=mt_split.info.AMR_AF[mt_split.a_index - 1],
    SAS_AF=mt_split.info.SAS_AF[mt_split.a_index - 1],
    DP=mt_split.info.DP,
    AA=mt_split.info.AA,
    VT=(hl.case()
       .when((mt_split.alleles[0].length() == 1) & (mt_split.alleles[1].length() == 1), 'SNP')
       .when(mt_split.alleles[0].matches('<CN*>') | mt_split.alleles[1].matches('<CN*>'), 'SV')
       .default('INDEL')),
    EX_TARGET=mt_split.info.EX_TARGET,
    MULTI_ALLELIC=mt_split.info.MULTI_ALLELIC))

n_rows, n_cols = mt_split.count()
n_partitions = mt_split.n_partitions()

mt_split = hl.sample_qc(mt_split)
mt_split = hl.variant_qc(mt_split)

mt_split = mt_split.annotate_globals(
    metadata=hl.struct(
        name='1000_Genomes_phase3_autosomes',
        reference_genome='GRCh37',
        n_rows=n_rows,
        n_cols=n_cols,
        n_partitions=n_partitions))

mt_split.write('gs://hail-datasets-hail-data/1000_Genomes_phase3_autosomes.GRCh37.mt', overwrite=True)

mt = hl.read_matrix_table('gs://hail-datasets-hail-data/1000_Genomes_phase3_autosomes.GRCh37.mt')
mt.describe()
print(mt.count())
""")

Example: We could infiltrate this...

ht_samples = hl.read_table('gs://hail-datasets-hail-data/1000_Genomes_phase3_samples.ht')
ht_relationships = hl.read_table('gs://hail-datasets-hail-data/1000_Genomes_phase3_sample_relationships.ht')

mt = hl.import_vcf(
 'gs://hail-datasets-raw-data/1000_Genomes/1000_Genomes_phase3_chr{1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22}_GRCh37.vcf.bgz',
 reference_genome='GRCh37')

mt = mt.annotate_cols(**ht_samples[mt.s])
mt = mt.annotate_cols(**ht_relationships[mt.s])

mt_split = hl.split_multi(mt)
mt_split = mt_split.select_entries(GT=hl.downcode(mt_split.GT, mt_split.a_index))
mt_split = mt_split.annotate_rows(info=hl.struct(
 CIEND=mt_split.info.CIEND[mt_split.a_index - 1],
 CIPOS=mt_split.info.CIPOS[mt_split.a_index - 1],
 CS=mt_split.info.CS,
 END=mt_split.info.END,
 IMPRECISE=mt_split.info.IMPRECISE,
 MC=mt_split.info.MC,
 MEINFO=mt_split.info.MEINFO,
 MEND=mt_split.info.MEND,
 MLEN=mt_split.info.MLEN,
 MSTART=mt_split.info.MSTART,
 SVLEN=mt_split.info.SVLEN,
 SVTYPE=mt_split.info.SVTYPE,
 TSD=mt_split.info.TSD,
 AC=mt_split.info.AC[mt_split.a_index - 1],
 AF=mt_split.info.AF[mt_split.a_index - 1],
 NS=mt_split.info.NS,
 AN=mt_split.info.AN,
 EAS_AF=mt_split.info.EAS_AF[mt_split.a_index - 1],
 EUR_AF=mt_split.info.EUR_AF[mt_split.a_index - 1],
 AFR_AF=mt_split.info.AFR_AF[mt_split.a_index - 1],
 AMR_AF=mt_split.info.AMR_AF[mt_split.a_index - 1],
 SAS_AF=mt_split.info.SAS_AF[mt_split.a_index - 1],
 DP=mt_split.info.DP,
 AA=mt_split.info.AA,
 VT=(hl.case()
 .when((mt_split.alleles[0].length() == 1) & (mt_split.alleles[1].length() == 1), 'SNP')
 .when(mt_split.alleles[0].matches('<CN*>') | mt_split.alleles[1].matches('<CN*>'), 'SV')
 .default('INDEL')),
 EX_TARGET=mt_split.info.EX_TARGET,
 MULTI_ALLELIC=mt_split.info.MULTI_ALLELIC))

n_rows, n_cols = mt_split.count()
n_partitions = mt_split.n_partitions()

mt_split = hl.sample_qc(mt_split)
mt_split = hl.variant_qc(mt_split)

mt_split = mt_split.annotate_globals(
 metadata=hl.struct(
 name='1000_Genomes_phase3_autosomes',
 reference_genome='GRCh37',
 n_rows=n_rows,
 n_cols=n_cols,
 n_partitions=n_partitions))

mt_split.write('gs://hail-datasets-hail-data/1000_Genomes_phase3_autosomes.GRCh37.mt', overwrite=True)

mt = hl.read_matrix_table('gs://hail-datasets-hail-data/1000_Genomes_phase3_autosomes.GRCh37.mt')
mt.describe()
print(mt.count())

In [119]:
# import logging
# import os
# import cloudstorage as gcs
# import webapp2
# from google.appengine.api import app_identity

In [120]:
# This notebook file is getting big, I'm going to split it into the next notebook... stay tuned...